In [1]:
import os
from importlib import import_module

from django import setup
from django.conf import settings

os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
my_settings = import_module("Configs.settings")
settings.configure(
    DATABASES=my_settings.DATABASES,
    INSTALLED_APPS=my_settings.INSTALLED_APPS,
    SECRET_KEY="django-insecure-&im0r5g-55jgm3r5y71n-)!_q#1s*1d0%en#1a(amut@332x5*",
)
setup()

from DB.models import AndroidMetadata, Label, Profile, RoomMasterTable, Session

Company Sum

In [ ]:
from jdatetime import datetime, date, time
from django.db.models import Sum

year, month, day = 1403, 7, 1
dt = datetime.combine(date(year, month, day), time(0, 0)).timestamp() * 1000
filters = {"timestamp__gte": dt, "label__title": "compony"}
filters

In [ ]:
list(Session.objects.filter(**filters))

In [ ]:
total = Session.objects.filter(**filters).aggregate(Sum("duration"))
total["duration__sum"] / 60 * 3 / 2

Mean 5 week work hours for rest of the week

In [ ]:
from jdatetime import date, datetime, time, timedelta

now = datetime.now()
weekday = now.weekday()
res = {}
for session in Session.objects.all().order_by("-timestamp"):
    if (
        session.gdatetime
        + timedelta(weeks=4)
        + timedelta(days=session.gdatetime.weekday() + 1)
        < now
    ):
        break
    if session.gdatetime.weekday() < weekday:
        continue
    weeknumber = session.gdatetime.isocalendar()[1]
    res[weeknumber] = res.get(weeknumber, 0) + session.duration

res_mean = round(sum(res.values()) / len(res))
f"{res_mean//60:02}:{res_mean%60:02}"